In [2]:
import soundfile as sf
import numpy as np
from numba import njit

In [3]:
import soundfile as sf
import numpy as np
from numba import njit


# ----------------------------------------------------------
# Comb filter (Schroeder): delay + feedback + lowpass
# ----------------------------------------------------------
@njit
def comb_filter(x, delay_samples, feedback, damp):
    buf = np.zeros(delay_samples, dtype=np.float32)
    y = np.zeros_like(x)
    idx = 0
    filter_state = 0.0

    for n in range(len(x)):
        bufout = buf[idx]
        # one-pole lowpass inside the loop
        filter_state = (1 - damp) * filter_state + damp * bufout

        y[n] = bufout
        buf[idx] = x[n] + filter_state * feedback

        idx = (idx + 1) % delay_samples

    return y


# ----------------------------------------------------------
# Allpass filter (diffusion)
# ----------------------------------------------------------
@njit
def allpass_filter(x, delay_samples, feedback):
    buf = np.zeros(delay_samples, dtype=np.float32)
    y = np.zeros_like(x)
    idx = 0

    for n in range(len(x)):
        bufout = buf[idx]
        input_sample = x[n]

        # classic allpass equation
        y[n] = -input_sample + bufout
        buf[idx] = input_sample + bufout * feedback

        idx = (idx + 1) % delay_samples

    return y


# ----------------------------------------------------------
# Full reverb mono → stereo
# ----------------------------------------------------------
def process_reverb(x, sr):
    # convert to mono if needed
    if x.ndim > 1:
        x = x.mean(axis=1)

    # length in seconds for delays
    comb_delays_ms = [30, 34, 40, 45]       # early reflection density
    ap_delays_ms   = [5, 1.7]               # diffusion

    # convert ms → samples
    comb_delays = [int(sr * d/1000) for d in comb_delays_ms]
    ap_delays   = [int(sr * d/1000) for d in ap_delays_ms]

    # PARAMETERS
    comb_feedback = 0.75
    comb_damp = 0.15
    ap_feedback = 0.6

    # run combs
    s = np.zeros_like(x)
    for d in comb_delays:
        s += comb_filter(x, d, comb_feedback, comb_damp)

    # run allpasses
    for d in ap_delays:
        s = allpass_filter(s, d, ap_feedback)

    # stereo output — simple decorrelation trick
    left  = s * 0.7
    right = s[::-1] * 0.7

    return np.column_stack((left, right))




In [7]:

if __name__ == "__main__":
    input_file = "/Users/marino/Documents/GitHub/fushimi-in-airy/reverb/thisIsAGoodExample.wav"
    output_file = "/Users/marino/Documents/GitHub/fushimi-in-airy/reverb/output_thisIsAGoodExample.wav"

    print("Loading:", input_file)
    x, sr = sf.read(input_file)

    print("Processing reverb...")
    y = process_reverb(x, sr)

    print("Saving:", output_file)
    sf.write(output_file, y, sr)

    print("Done! Listen to:", output_file)

Loading: /Users/marino/Documents/GitHub/fushimi-in-airy/reverb/thisIsAGoodExample.wav
Processing reverb...
Saving: /Users/marino/Documents/GitHub/fushimi-in-airy/reverb/output_thisIsAGoodExample.wav
Done! Listen to: /Users/marino/Documents/GitHub/fushimi-in-airy/reverb/output_thisIsAGoodExample.wav
